In [1]:
import pandas as pd

In [2]:
# loading the data
df = pd.read_csv("loan.csv")
df.shape
df_clean = df.copy()

/var/folders/wn/wpy04pb15y934q92j5gbcvb80000gn/T/ipykernel_97527/2379315871.py:2: DtypeWarning: Columns (19,47,55,112,123,124,125,128,129,130,133,139,140,141) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("loan.csv")


In [3]:
# Keep only relevant rows
df_clean = df[df['loan_status'].isin(['Fully Paid', 'Charged Off', 'Default'])].copy()

# Convert dates once (vectorized)
df_clean['issue_d'] = pd.to_datetime(df_clean['issue_d'], format='%b-%Y', errors='coerce')
df_clean['last_pymnt_d'] = pd.to_datetime(df_clean['last_pymnt_d'], format='%b-%Y', errors='coerce')

# Extract term in months
df_clean['term_months'] = df_clean['term'].str.extract(r'(\d+)').astype(float)

# Expected loan end date
df_clean['expected_end'] = df_clean['issue_d'] + pd.to_timedelta(df_clean['term_months'] * 30, unit='D')

# Flag late fully paid
df_clean['paid_late_flag'] = (
    (df_clean['loan_status'] == 'Fully Paid') &
    (df_clean['last_pymnt_d'] > df_clean['expected_end'])
)

# Build 3-class target
df_clean['target_3class'] = 'paid_on_time'
df_clean.loc[df_clean['paid_late_flag'], 'target_3class'] = 'paid_late'
df_clean.loc[df_clean['loan_status'].isin(['Charged Off', 'Default']), 'target_3class'] = 'not_paid'

# Check distribution
print(df_clean['target_3class'].value_counts())


target_3class
paid_on_time    831511
not_paid        261686
paid_late       210441
Name: count, dtype: int64


In [4]:
# כמה ערכים חסרים?
missing_count = df_clean['target_3class'].isna().sum()

# כמה סה״כ שורות יש?
total_rows = len(df)

# אחוז חסרים
missing_percent = (missing_count / total_rows) * 100

print(f"Missing target_3class: {missing_count} rows")
print(f"Total rows: {total_rows}")
print(f"Percentage missing: {missing_percent:.4f}%")


Missing target_3class: 0 rows
Total rows: 2260668
Percentage missing: 0.0000%


In [5]:
def minimal_clean1(df: pd.DataFrame) -> pd.DataFrame:
    clean_df = df.copy()

    # --- 1. Remove leakage columns ---
    leakage_columns = [
        'hardship_flag', 'debt_settlement_flag',
        'total_pymnt', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee',
        'last_pymnt_d', 'last_pymnt_amnt', 'recoveries',
        'collection_recovery_fee', 'out_prncp', 'total_pymnt_inv', 'out_prncp_inv'
    ]
    clean_df = clean_df.drop(columns=[c for c in leakage_columns if c in clean_df.columns])

    # --- 2. Remove columns with > 90% missing ---
    missing_percent = clean_df.isnull().mean() * 100
    high_missing_cols_90 = missing_percent[missing_percent > 90].index.tolist()
    clean_df = clean_df.drop(columns=high_missing_cols_90)

    # --- 3. Smart high-correlation removal (>0.95) ---
    # only numerical columns
    num_cols = clean_df.select_dtypes(include='number').columns

    if len(num_cols) > 1:
        corr = clean_df[num_cols].corr().abs()
        missing = clean_df[num_cols].isnull().mean()
        var = clean_df[num_cols].var()

        to_drop = set()

        for i, c1 in enumerate(num_cols):
            for j, c2 in enumerate(num_cols):
                if j <= i:
                    continue
                if corr.loc[c1, c2] > 0.95:

                    if c1 in to_drop or c2 in to_drop:
                        continue

                    # choose worst feature
                    if missing[c1] > missing[c2]:
                        to_drop.add(c1)
                    elif missing[c2] > missing[c1]:
                        to_drop.add(c2)
                    else:
                        drop_col = c1 if var[c1] < var[c2] else c2
                        to_drop.add(drop_col)

        clean_df = clean_df.drop(columns=list(to_drop))

    return clean_df


df_clean = minimal_clean1(df_clean)

In [6]:
def minimal_clean2(df: pd.DataFrame) -> pd.DataFrame:
    clean_df = df.copy()

    # --- 1. term -> turn into numeric (36 / 60)
    clean_df['term'] = (clean_df['term'].astype(str).str.extract(r'(\d+)')[0].astype(float))

    # --- 2. emp_length -> turn into numeric years
    emp = clean_df['emp_length'].astype(str)
    emp = emp.str.replace('< 1', '0', regex=False)
    emp = emp.str.extract(r'(\d+)')[0]
    clean_df['emp_length'] = emp.astype(float)

    # --- 3. issue_d -> optional: extract year, but DO NOT drop original col
    clean_df['issue_year'] = (clean_df['issue_d'].astype(str).str.extract(r'(\d{4})')[0].astype(float))
    # לא מוחקים את issue_d בשלב המינימלי

    return clean_df

df_clean = minimal_clean2(df_clean)

In [7]:
print(len(df_clean.columns))
print(df_clean.columns)

91
Index(['loan_amnt', 'term', 'int_rate', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'issue_d', 'loan_status', 'pymnt_plan', 'purpose', 'title', 'zip_code',
       'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record',
       'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'initial_list_status', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'application_type', 'acc_now_delinq', 'tot_coll_amt',
       'open_acc_6m', 'open_act_il', 'open_il_12m', 'open_il_24m',
       'mths_since_rcnt_il', 'il_util', 'open_rv_12m', 'open_rv_24m',
       'max_bal_bc', 'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl',
       'inq_last_12m', 'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy',
       'bc_util', 'chargeoff_within_12_mths', 'delinq_amnt',
     